# Notebook 1: Data Tasks (Part 1)

In this notebook, we will implement an example solution for computing how often products co-occur in shopping baskets.

In [1]:
import os

import numpy as np
import pandas as pd
import scipy.sparse
import tqdm

In [2]:
# please update me!
PATH_DATA = "../data/instacart"

<br> 
<br> 

## Preparation: Get the data

### Download data from Kaggle

1. Create account on www.kaggle.com
1. Download the Instacart data set from kaggle.com: https://www.kaggle.com/c/instacart-market-basket-analysis/data
1. Put your data into `PATH_DATA` (as specified above) and unzip files

In [3]:
# check that the required files are available
assert os.path.isfile(f"{PATH_DATA}/orders.csv")
assert os.path.isfile(f"{PATH_DATA}/order_products__prior.csv")
assert os.path.isfile(f"{PATH_DATA}/order_products__train.csv")

### Load data

In [4]:
orders = pd.read_csv(f"{PATH_DATA}/orders.csv")
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [5]:
order_products = pd.concat(
    [
        pd.read_csv(f"{PATH_DATA}/order_products__prior.csv"),
        pd.read_csv(f"{PATH_DATA}/order_products__train.csv"),
    ]
)
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


<br>
<br>

## Test data

First, we create some test data and a reference result. The reference solution is created in a very naïve way that is bullet proof; not efficient but sufficient for small data. We can compare proper solutions with this "ground truth."

In [6]:
np.random.seed(123)

In [7]:
n_baskets = 8
n_products = 8
max_n_products = 4

In [8]:
basket_list = []
for _ in range(n_baskets):
    size_b = np.random.choice(range(2, max_n_products))
    basket_list.append(np.random.choice(n_products, size_b, replace=False))
basket_list

[array([0, 1]),
 array([7, 6, 4]),
 array([5, 4, 2]),
 array([5, 6]),
 array([6, 7, 1]),
 array([6, 0]),
 array([1, 4]),
 array([7, 0])]

In [9]:
basket_df = pd.DataFrame(
    {
        "basket": np.repeat(range(len(basket_list)), [len(x) for x in basket_list]),
        "product": [y for x in basket_list for y in x],
    }
)
basket_df.head()

,basket,product
0,0,0
1,0,1
2,1,7
3,1,6
4,1,4


In [10]:
truth = np.zeros((n_products, n_products), dtype=int)

for b in basket_list:
    for j in b:
        for i in b:
            truth[i, j] += 1

# diagonal is the count of baskets that products occur in
assert np.all(
    basket_df.sort_values("product").groupby("product").basket.count().values
    == np.diag(truth)[np.diag(truth) > 0]
)

# matrix is symmetric
assert np.all(truth == truth.T)

truth

array([[3, 1, 0, 0, 0, 0, 1, 1],
       [1, 3, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 3, 1, 1, 1],
       [0, 0, 1, 0, 1, 2, 1, 0],
       [1, 1, 0, 0, 1, 1, 4, 2],
       [1, 1, 0, 0, 1, 0, 2, 3]])

## Computation of co-occurrences using list

In [11]:
# turn the input data frame into a list of baskets, each basket is a list of products
# slightly more than we need now, but we'll see this function again in one of the future exercises
def baskets_df_to_list(
    x,
    variable_basket="basket",
    variable_product="product",
    min_basket_size=0,
    shuffle=False,
    to_string=False,
    seed=501,
):

    # create raw basket list
    x_basket_product = x[[variable_basket, variable_product]]
    n_products = x[variable_product].max() + 1
    keys, values = x_basket_product.sort_values(variable_basket).values.T
    ukeys, index = np.unique(keys, True)
    arrays = np.split(values, index)
    basket_list = [list(set(a)) for a in arrays[1:]]

    # format basket list
    basket_list_out = []
    for basket in basket_list:
        if len(basket) >= min_basket_size:
            if to_string:
                basket_list_out.append([str(x) for x in basket])
            else:
                basket_list_out.append(basket)

    # randomize basket order and product order in baskets
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(basket_list_out)
        for i in range(len(basket_list_out)):
            np.random.shuffle(basket_list_out[i])

    return basket_list_out, n_products


# for each basket increase the co-occurrence counts for the products in the basket by 1
def co_occurrences_list(x, variable_basket="basket", variable_product="product"):
    basket_list, n_products = baskets_df_to_list(
        x=x,
        variable_basket=variable_basket,
        variable_product=variable_product,
        min_basket_size=0,
        shuffle=False,
        to_string=False,
    )

    co_occurrences = np.zeros((n_products, n_products), dtype=int)
    for b in tqdm.tqdm(basket_list):
        co_occurrences[np.ix_(b, b)] += 1

    return co_occurrences

In [12]:
co_occurrences = co_occurrences_list(basket_df)
assert np.all(truth == co_occurrences)
co_occurrences

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 5259.32it/s]


array([[3, 1, 0, 0, 0, 0, 1, 1],
       [1, 3, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 3, 1, 1, 1],
       [0, 0, 1, 0, 1, 2, 1, 0],
       [1, 1, 0, 0, 1, 1, 4, 2],
       [1, 1, 0, 0, 1, 0, 2, 3]])

## Computation of co-occurrences using sparse matrix

In [13]:
# create a (binary) space matrix that indicates whether a basket (row) contains a product (col)
# co-occurrences are the dot produdct of this basket-product matrix
def co_occurrences_sparse(x, variable_basket="basket", variable_product="product"):
    row = x[variable_basket].values
    col = x[variable_product].values
    dim = (x[variable_basket].max() + 1, x[variable_product].max() + 1)

    basket_product_table = scipy.sparse.csr_matrix(
        (np.ones(len(row), dtype=int), (row, col)), shape=dim
    )
    co_occurrences_sparse = basket_product_table.T.dot(basket_product_table)
    co_occurrences_dense = co_occurrences_sparse.toarray()
    return co_occurrences_dense

In [14]:
co_occurrences_2 = co_occurrences_sparse(basket_df)
assert np.all(truth == co_occurrences_2)
co_occurrences_2

array([[3, 1, 0, 0, 0, 0, 1, 1],
       [1, 3, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 3, 1, 1, 1],
       [0, 0, 1, 0, 1, 2, 1, 0],
       [1, 1, 0, 0, 1, 1, 4, 2],
       [1, 1, 0, 0, 1, 0, 2, 3]])

<br>
<br>

## 1. &ensp; Task 1: "Product co-occurrence"

For all products (`product_id`), compute how often the product co-occurs in orders (`order_id`) with every other product. The output should be a `pd.DataFrame` with the following three columns:

1. Product 1
1. Product 2
1. Number of times the products co-occur in a shopping basket

In [15]:
co_occurrences = co_occurrences_list(
    x=order_products,
    variable_basket="order_id",
    variable_product="product_id",
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3346083/3346083 [08:20<00:00, 6685.25it/s]


In [16]:
co_occurrences_2 = co_occurrences_sparse(
    x=order_products,
    variable_basket="order_id",
    variable_product="product_id",
)

In [17]:
# only use a (random) subset of product pairs so the comparison is quicker
idx = np.random.choice(co_occurrences_2.shape[0], 5_000)
assert np.all(co_occurrences[np.ix_(idx, idx)] == co_occurrences_2[np.ix_(idx, idx)])

<br>
<br>

## Some time measurements

In [18]:
%load_ext line_profiler

In [19]:
# only use a subset of data
order_products_sample = order_products[order_products["order_id"] < 10_000]

In [20]:
%lprun -f co_occurrences_list co_occurrences_list(x=order_products_sample, variable_basket="order_id", variable_product="product_id")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9812/9812 [00:02<00:00, 4542.83it/s]


Timer unit: 1e-06 s

Total time: 2.35764 s
File: /var/folders/jp/gv12f6ys7_nfjtp_8wr77vpr0000gn/T/ipykernel_38467/1180061722.py
Function: co_occurrences_list at line 41

Line #      Hits         Time  Per Hit   % Time  Line Contents
    41                                           def co_occurrences_list(x, variable_basket="basket", variable_product="product"):
    42         2     193446.0  96723.0      8.2      basket_list, n_products = baskets_df_to_list(
    43         1          1.0      1.0      0.0          x=x,
    44         1          1.0      1.0      0.0          variable_basket=variable_basket,
    45         1          1.0      1.0      0.0          variable_product=variable_product,
    46         1          1.0      1.0      0.0          min_basket_size=0,
    47         1          1.0      1.0      0.0          shuffle=False,
    48         1          0.0      0.0      0.0          to_string=False,
    49                                               )
    50          

In [21]:
%lprun -f co_occurrences_sparse co_occurrences_sparse(x=order_products_sample, variable_basket="order_id", variable_product="product_id")

Timer unit: 1e-06 s

Total time: 1.02142 s
File: /var/folders/jp/gv12f6ys7_nfjtp_8wr77vpr0000gn/T/ipykernel_38467/2429178872.py
Function: co_occurrences_sparse at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def co_occurrences_sparse(x, variable_basket="basket", variable_product="product"):
     4         1        749.0    749.0      0.1      row = x[variable_basket].values
     5         1         26.0     26.0      0.0      col = x[variable_product].values
     6         1        932.0    932.0      0.1      dim = (x[variable_basket].max() + 1, x[variable_product].max() + 1)
     7                                           
     8         2      10473.0   5236.5      1.0      basket_product_table = scipy.sparse.csr_matrix(
     9         1        166.0    166.0      0.0          (np.ones(len(row), dtype=int), (row, col)), shape=dim
    10                                               )
    11         1      34

## Pure sparse implementation

Most of the time is spent on turning the sparse matrix into a regular numpy array. Instead, we can keep the data sparse by turning the sparse matrix into a pandas data frame. This will significantly speed up the computation.

#### Implement sparse calculation, return pandas data frame

In [22]:
# create a (binary) space matrix that indicates whether a basket (row) contains a product (col)
# co-occurrences are the dot produdct of this basket-product matrix
# return data frame
def co_occurrences_sparse_2(x, variable_basket="basket", variable_product="product"):
    row = x[variable_basket].values
    col = x[variable_product].values
    dim = (x[variable_basket].max() + 1, x[variable_product].max() + 1)

    basket_product_table = scipy.sparse.csr_matrix(
        (np.ones(len(row), dtype=int), (row, col)), shape=dim
    )
    co_occurrences_sparse = basket_product_table.T.dot(basket_product_table).tocoo()
    co_occurrences_df = pd.DataFrame(
        {
            "product_1": co_occurrences_sparse.row,
            "product_2": co_occurrences_sparse.col,
            "co-occurrence": co_occurrences_sparse.data,
        }
    )
    return co_occurrences_df

In [23]:
%lprun -f co_occurrences_sparse_2 co_occurrences_sparse_2(x=order_products_sample, variable_basket="order_id", variable_product="product_id")

Timer unit: 1e-06 s

Total time: 0.064485 s
File: /var/folders/jp/gv12f6ys7_nfjtp_8wr77vpr0000gn/T/ipykernel_38467/3289034551.py
Function: co_occurrences_sparse_2 at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def co_occurrences_sparse_2(x, variable_basket="basket", variable_product="product"):
     5         1         47.0     47.0      0.1      row = x[variable_basket].values
     6         1         16.0     16.0      0.0      col = x[variable_product].values
     7         1        719.0    719.0      1.1      dim = (x[variable_basket].max() + 1, x[variable_product].max() + 1)
     8                                           
     9         2       5671.0   2835.5      8.8      basket_product_table = scipy.sparse.csr_matrix(
    10         1        316.0    316.0      0.5          (np.ones(len(row), dtype=int), (row, col)), shape=dim
    11                                               )
    12         1   

#### Produce result

In [24]:
co_occurrences_df = co_occurrences_sparse_2(
    x=order_products_sample,
    variable_basket="order_id",
    variable_product="product_id",
)
co_occurrences_df = co_occurrences_df.sort_values(
    ["product_1", "product_2"]
).reset_index(drop=True)
co_occurrences_df

,product_1,product_2,co-occurrence
0,1,1,7
1,1,130,1
2,1,196,1
3,1,769,1
4,1,3298,1
...,...,...,...
1165846,49685,30489,1
1165847,49685,34939,1
1165848,49685,49685,1
1165849,49686,12204,1


In [25]:
%timeit -n 1 co_occurrences_sparse_2(x=order_products, variable_basket="order_id", variable_product="product_id")

20.1 s ± 2.39 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


That is much(!) faster than the old sparse implementation and the list implementation.

#### Check whether the results is correct

In [26]:
reference_result = co_occurrences_list(
    x=order_products_sample,
    variable_basket="order_id",
    variable_product="product_id",
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9812/9812 [00:01<00:00, 6736.95it/s]


In [27]:
matrix_from_co_occurrences_df = np.zeros(reference_result.shape)
matrix_from_co_occurrences_df[
    (co_occurrences_df["product_1"], co_occurrences_df["product_2"])
] = co_occurrences_df["co-occurrence"].values
assert np.all(reference_result == matrix_from_co_occurrences_df)

<br>
<br>
&mdash; <br>
Sebastian Gabel <br>
`Learning from Big Data` <br>